## TensorFlow Guide
### Introduction
https://www.tensorflow.org/guide/low_level_intro 

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) #tf.float32 implicitly

total = a + b
print(a)
print(b)
print(total)

Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [5]:
sess = tf.Session()
print(sess.run(total))

7.0


In [6]:
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


**One value within a session**

e.g. the two ***sess.run(vec)*** calls gives two sets of values for vec.
That's because these were two run calls. If they were in one then
vec would have only one value like shown in ***sess.run((out, out2))*** call

In [70]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))
print(sess.run(out1))


[0.06374729 0.688208   0.7294078 ]
[0.95203805 0.6742761  0.36814666]
(array([1.0235859, 1.3800303, 1.0080912], dtype=float32), array([2.0235858, 2.3800302, 2.0080912], dtype=float32))
[1.3133479 1.5018744 1.7130816]


In [71]:
print(sess.run(out1))

[1.334705 1.273536 1.857341]


**Feeding**

In [36]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

print(z)

Tensor("add_4:0", dtype=float32)


In [38]:
print(sess.run(z, feed_dict={x: 2, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))
print(sess.run(z, {x: 4, y: 5}))

6.5
[3. 7.]
9.0


**Datasets**

In [13]:
my_data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [25]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

In [31]:
sess.run(iterator.initializer)

In [32]:
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[ 0.22501922 -2.0728457  -0.725248  ]
[ 1.8925657  -0.13847487  0.07284713]
[ 0.68078023 -0.06069195  0.23558158]
[ 0.5102637  -0.47065938 -0.5242628 ]
[-0.61051047 -0.82380474 -1.1289504 ]
[0.23460358 0.56900674 0.91986287]
[-1.8023225 -1.7104388  0.1892082]
[0.7863023  0.48373973 0.16975455]
[-1.9684476  1.006028  -1.5054568]
[ 0.0054974  -1.7558231  -0.41747016]


**Layers**

In [43]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [44]:
init = tf.global_variables_initializer()
sess.run(init)

In [46]:
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.195876  ]
 [ 0.40245104]]


*Layer Function Shortcuts*

In [42]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[3.0949063]
 [7.3956013]]


**Features**

In [47]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [48]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [49]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


**Training**

In [50]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [51]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [61]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.3871846]
 [-2.7743692]
 [-4.161554 ]
 [-5.5487385]]


In [72]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

4.0602627


In [74]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [75]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

4.0602627
2.8476934
2.0061355
1.4220161
1.0165285
0.7349911
0.5394608
0.40361035
0.30917138
0.24346823
0.1977051
0.16577902
0.1434552
0.1277953
0.11676026
0.10893537
0.10333889
0.09928974
0.09631518
0.09408726
0.09237835
0.091030635
0.08993447
0.08901378
0.08821586
0.087504044
0.08685289
0.08624485
0.085667595
0.085112676
0.08457415
0.08404791
0.08353113
0.08302179
0.08251854
0.08202041
0.08152673
0.08103699
0.080550894
0.08006816
0.07958867
0.079112306
0.07863893
0.07816853
0.07770097
0.07723629
0.07677442
0.07631533
0.075858995
0.07540545
0.074954584
0.07450642
0.07406098
0.073618144
0.07317799
0.07274047
0.07230557
0.071873255
0.07144356
0.071016386
0.0705918
0.07016974
0.06975023
0.069333196
0.06891865
0.0685066
0.068096995
0.067689896
0.067285165
0.066882886
0.066483
0.06608549
0.065690406
0.065297626
0.064907216
0.06451916
0.06413341
0.06374997
0.06336881
0.06298995
0.06261333
0.062238976
0.061866857
0.061496966
0.061129283
0.060763806
0.06040049
0.06003938
0.059680395
0.05932360

*Complete Program*

In [77]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))

0.79425657
0.592605
0.45243546
0.35492805
0.28702447
0.23966414
0.20655963
0.18334837
0.16700326
0.15542388
0.14715277
0.14117864
0.1367997
0.13352905
0.13102879
0.12906444
0.12747337
0.12614265
0.12499393
0.12397285
0.123041674
0.12217425
0.121352315
0.12056333
0.11979846
0.11905168
0.1183187
0.117596544
0.116883256
0.116177335
0.11547785
0.11478401
0.11409536
0.11341154
0.11273236
0.11205753
0.111387014
0.11072069
0.11005843
0.10940024
0.10874603
0.10809575
0.1074494
0.10680693
0.10616833
0.10553352
0.10490255
0.10427538
0.10365192
0.10303218
0.10241614
0.101803854
0.10119514
0.10059014
0.099988714
0.09939092
0.09879669
0.098205976
0.0976188
0.09703518
0.09645501
0.0958783
0.09530509
0.09473525
0.09416884
0.093605824
0.09304617
0.092489876
0.09193689
0.09138721
0.09084081
0.090297714
0.08975782
0.08922118
0.08868774
0.08815749
0.0876304
0.0871065
0.08658568
0.08606801
0.08555343
0.0850419
0.08453346
0.08402805
0.08352566
0.083026245
0.082529865
0.08203645
0.08154593
0.0810584
0.08057